# Lemma Candidates

In [1]:
import numpy as np
import pandas as pd
import re
import multiprocessing as mp
from tqdm.auto import tqdm
from tqdm.contrib import tzip
import spacy
import pickle
from string import punctuation
from spacy.lang.en import stop_words

2024-10-24 19:49:37.896914: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 19:49:37.896978: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 19:49:37.901584: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-24 19:49:37.920273: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-24 19:49:39.103015: W tensorflow/compiler/tf2

In [2]:
class TextProcessor():
    def __init__(self, kn_base='en_core_web_sm'):
        self.nlp = spacy.load(kn_base)
        self.stop_wrods = stop_words.STOP_WORDS | set(punctuation) | set('123456789')
        
    def prepare_text(self, s):
        """Базовая предобработка текста перед лемматизацией"""
        s = s.lower().replace('_',' ').strip()
        for p in punctuation:
            s = s.replace(p,' ')
        return re.sub(" +", " ",s)
    
    def precess_text(self,s):
        """Лемматизируем запрос и удаляем стоп слова"""
        s = self.prepare_text(s)
        doc = self.nlp(s)
        lemms = [token.lemma_ for token in doc]
        lemms = [token for token in lemms if token not in self.stop_wrods]
        return lemms
    
    def process_batch(self,texts,verbose=True):
        """Для увеличения скорости паралелим вычесления"""
        iterator = tqdm(texts) if verbose else texts
        pool = mp.Pool(mp.cpu_count())
        prep_texts = pool.map(self.precess_text,iterator)
        return prep_texts

In [3]:
class EntityLinlker():
    def __init__(self, kg, text_proc, prepared2concept):
        self.kg = kg
        self.text_proc = text_proc
        self.p2c = prepared2concept
        
    def link_sentence(self,s):
        """Ищем для запроса релевантных кандидатов"""
        candidates = []
        s = self.text_proc.precess_text(s)
        s_st = set(s)
        for i,k in enumerate(self.kg):
            if len(k) == 0:
                continue
            elif len(k) == 1:
                if k[0] in s_st:
                    candidates.append(i)
            else:
                if all(x in s_st for x in k):
                    idxes = [s.index(x) for x in k]
                    if all(idxes[j+1] - idxes[j] == 1 for j in range(len(k)-1)):
                           candidates.append(i)
        return candidates
    
    def batch_linking(self,texts,verbose=True):
        """Для увеличения скорости паралелим вычесления"""
        iterator = tqdm(texts) if verbose else texts
        pool = mp.Pool(mp.cpu_count())
        prep_texts = pool.map(self.link_sentence,iterator)
        return prep_texts

In [4]:
processor = TextProcessor()
graph_state = pickle.load(open('graph.pkl','rb'))
concepts = graph_state['concepts_maper'].keys()
concepts = [(x if type(x) == str else '') for x in concepts] # удаляем валидные вершины
prepared_concepts =  processor.process_batch(concepts) # pickle.load(open('conceprs_enc.pkl','rb')) - если не хотите ждать

  0%|          | 0/799272 [00:00<?, ?it/s]

In [5]:
def texual_acc(s1,s2): # like bleu@1 score
    return sum(i == j for i,j in zip(s1,s2)) / (max(len(s1),len(s2)) + 1e-6)

def prep_text(s):
    return s.lower().strip().replace('_', ' ')

def rank_func(s1,texts):
    s1 = prep_text(s1)
    scores = []
    for s in texts:
        scores.append(texual_acc(s1,prep_text(s)))
    return texts[np.argmax(scores)]

In [6]:
# из за лемматизации, текстовая составляющая некоторых вершин стала идентичной
# по этому для их текста ищем вершину, чья лемматезированная версия как можно сильнее совподает с не лемматезированной
# таким образом получаем словарь, который сопоставляет лемматизированному концепту его исходную версию
concept2prepared = dict(zip(concepts,prepared_concepts))
df_cp = pd.DataFrame(zip(concepts,prepared_concepts))
df_cp[1] = df_cp[1].apply(lambda x: ' '.join(x))
df_cp_grp = df_cp.groupby(1)[0].agg(list)

In [7]:
ranked_nodes = [rank_func(y,x) for x,y in tzip(df_cp_grp.tolist(), df_cp_grp.index.tolist())]
prepared2concept = dict(zip(df_cp_grp.index.tolist(),ranked_nodes))

  0%|          | 0/656261 [00:00<?, ?it/s]

In [8]:
def parse_dialog(sentences):
    """Функция для предобработки диалога"""
    return '; '.join(sentences)

def parse_request(x):
    """Функция, создающая итоговый запрос"""
    return f"{x['dialog']} {x['question']} {x['choice_0']} {x['choice_1']}  {x['choice_2']}"
    

def make_df(path, is_test=False):
    """Функция предобработки данных для Dream"""
    data = pd.read_json(path,lines=True)
    df = []
    for i in range(len(data)):
        row = data.iloc[i]
        dialog = parse_dialog(row[0])
        for ques in row[1]:
            df.append({
                'dialog_id': i,
                'dialog': dialog,
                'question': ques['question'],
                'choice_0': ques['choice'][0],
                'choice_1': ques['choice'][1],
                'choice_2': ques['choice'][2],
                'labels': -100 if is_test else ques['choice'].index(ques['answer'])
            })
    return pd.DataFrame(df)

In [9]:
# Загружаем данные
train_data = make_df('data/train.statement.jsonl')
vall_data = make_df('data/dev.statement.jsonl')
test_data = make_df('data/test.statement.jsonl',is_test=True)

In [10]:
# Сопоставляем запросы
train_data['request'] = train_data.apply(parse_request,axis=1)
vall_data['request'] = vall_data.apply(parse_request,axis=1)
test_data['request'] = test_data.apply(parse_request,axis=1)

In [11]:
linker = EntityLinlker(prepared_concepts, processor, prepared2concept)

In [12]:
# ищем кандидатов
train_data['candidates'] = linker.batch_linking(train_data['request'].tolist())
vall_data['candidates'] = linker.batch_linking(vall_data['request'].tolist())
test_data['candidates'] = linker.batch_linking(test_data['request'].tolist())

  0%|          | 0/7307 [00:00<?, ?it/s]

  0%|          | 0/813 [00:00<?, ?it/s]

  0%|          | 0/2077 [00:00<?, ?it/s]

In [13]:
def get_concepts(ids):
    """Преобразуем концепты в число"""
    ids = [prepared_concepts[i] for i in ids]
    ids = [' '.join(x) for x in ids]
    return [prepared2concept[i] for i in ids]

In [14]:
test_data['candidates'] = test_data['candidates'].apply(get_concepts)
train_data['candidates'] = train_data['candidates'].apply(get_concepts)
vall_data['candidates'] = vall_data['candidates'].apply(get_concepts)

In [15]:
idf_matrix = pd.Series(
    np.concatenate(test_data['candidates'].tolist() + train_data['candidates'].tolist() + vall_data['candidates'].tolist())
).value_counts()
idf_matrix = idf_matrix.to_dict()

In [16]:
with open('idf_dream.pkl','wb') as f:
    pickle.dump(idf_matrix,f)

In [17]:
# Берём семплы с наименьшей популярностью, так как они будут более значимыми для нашей модели
# Таким образом получается получить top-k самых редких семплов, о которых исходная LM скорее всего не обладает знаниями
def rank_by_idf(texts,topk=5):
    idx = np.unique(texts, return_index=True)[1]
    texts = [texts[i] for i in sorted(idx)]
    idfs = [idf_matrix[x] for x in texts]
    return [texts[i] for i in np.argsort(idfs)[:topk]]

In [18]:
test_data['candidates_rank'] = test_data['candidates'].apply(rank_by_idf)
train_data['candidates_rank'] = train_data['candidates'].apply(rank_by_idf)
vall_data['candidates_rank'] = vall_data['candidates'].apply(rank_by_idf)

In [19]:
test_data['candidates_rank'].to_csv('./prep_data/test_rank_by_ids.csv',index=False)
train_data['candidates_rank'].to_csv('./prep_data/train_rank_by_ids.csv',index=False)
vall_data['candidates_rank'].to_csv('./prep_data/vall_rank_by_ids.csv',index=False)

# BGE Canidates Search

## Candidates from request

In [20]:
import pandas as pd
import numpy as np
from transformers import AutoModel, AutoTokenizer
import faiss
import torch
from tqdm.auto import trange
import pickle
import re
import torch.functional as F

In [21]:
def parse_dialog(sentences):
    """Функция для предобработки диалога"""
    return '; '.join(sentences)

def prepare_text(s):
    """Функция, создающая итоговый запрос"""
    return re.sub(" +", " ",s.lower().replace('_',' ').strip())

def parse_request(x):
    """Функция, создающая итоговый запрос"""
    return f"{x['dialog']} {x['question']} {x['choice_0']} {x['choice_1']}  {x['choice_2']}"

def make_df(path, is_test=False):
    """Функция предобработки данных для Dream"""
    data = pd.read_json(path,lines=True)
    df = []
    
    for i in range(len(data)):
        row = data.iloc[i]
        dialog = parse_dialog(row[0])
        for ques in row[1]:
            df.append({
                'dialog_id': i,
                'dialog': dialog,
                'question': ques['question'],
                'choice_0': ques['choice'][0],
                'choice_1': ques['choice'][1],
                'choice_2': ques['choice'][2],
                'labels': -100 if is_test else ques['choice'].index(ques['answer'])
            })
    return pd.DataFrame(df)

In [22]:
# Загружаем данные
train_data = make_df('data/train.statement.jsonl')
vall_data = make_df('data/dev.statement.jsonl')
test_data = make_df('data/test.statement.jsonl',is_test=True)

In [23]:
# Сопоставляем запросы
train_data['request'] = train_data.apply(parse_request,axis=1)
vall_data['request'] = vall_data.apply(parse_request,axis=1)
test_data['request'] = test_data.apply(parse_request,axis=1)

In [24]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
def cls_pooling(model_output):
    return model_output[0][:, 0]

@torch.no_grad()
def create_embeds(texts,model,tokenizer,batch_size=64,max_length=1024):
    """Функция, для получения векторных представлений из текста"""
    features = []
    texts = [prepare_text(x) for x in texts]
    for i in trange(len(texts) // batch_size + 1):
        encs = tokenizer(
            texts[i*batch_size:(i+1)*batch_size],
            truncation=True,
            padding='longest',
            max_length=max_length,
            return_tensors='pt'
        )
        encs['input_ids'] = encs['input_ids'].cuda()
        encs['attention_mask'] = encs['attention_mask'].cuda()
        if 'token_type_ids' in encs.keys():
            encs['token_type_ids'] = encs['token_type_ids'].cuda()
        model_output = model(**encs)
        embeds = cls_pooling(model_output)#mean_pooling(model_output, encs['attention_mask'])
        embeds = torch.nn.functional.normalize(embeds, p=2, dim=1).detach().cpu().numpy()
        features.append(embeds)
    return np.concatenate(features,dtype=np.float32)

In [25]:
# Загружаем retriveal модель
encoder = AutoModel.from_pretrained('BAAI/bge-small-en-v1.5').cuda()
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-small-en-v1.5')

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [26]:
graph_state = pickle.load(open('graph.pkl','rb'))
concepts = [prepare_text(s)  if type(s) == str else ' ' for s in graph_state['concepts_maper']] # делаем препроцессинг вершин

In [27]:
with open('cleared_concepts.pkl','wb') as f:
    pickle.dump(concepts,f)

In [28]:
# получаем векторные представления вершин
features = create_embeds(concepts,encoder,tokenizer,batch_size=512)

  0%|          | 0/1562 [00:00<?, ?it/s]

In [29]:
# Создаём faiss index для быстрого поиска кандидатов
index = faiss.IndexFlatIP(features.shape[1]) #use faiss.IndexFlatL2 for l2 distance
faiss.normalize_L2(features)
index.add(features)
res = faiss.StandardGpuResources()
index = faiss.index_cpu_to_gpu(res, 0, index)
index.ntotal

799272

In [30]:
# Получаем векторные представления запросов
train_features = create_embeds(train_data['request'].tolist(),encoder,tokenizer,batch_size=128,max_length=512)
test_features = create_embeds(test_data['request'].tolist(),encoder,tokenizer,batch_size=128,max_length=512)
val_features = create_embeds(vall_data['request'].tolist(),encoder,tokenizer,batch_size=128,max_length=512)

  0%|          | 0/58 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [31]:
faiss.normalize_L2(train_features)
D_train,I_train = index.search(train_features,k=5)
faiss.normalize_L2(test_features)
D_test,I_test = index.search(test_features,k=5)
faiss.normalize_L2(val_features)
D_vall,I_vall = index.search(val_features,k=5)

In [32]:
train_data['candidates'] = [[concepts[j] for j in x] for x in I_train]
test_data['candidates'] = [[concepts[j] for j in x] for x in I_test]
vall_data['candidates'] = [[concepts[j] for j in x] for x in I_vall]

In [33]:
train_data['candidates'].to_csv('./prep_data/train_dream_bge.csv',index=False)
test_data['candidates'].to_csv('./prep_data/test_dream_bge.csv',index=False)
vall_data['candidates'].to_csv('./prep_data/val_dream_bge.csv',index=False)

## Choice Encoding

In [34]:
train_choices = train_data['choice_0'].tolist() + train_data['choice_1'].tolist() + train_data['choice_2'].tolist()
test_choices = test_data['choice_0'].tolist() + test_data['choice_1'].tolist() + test_data['choice_2'].tolist()
vall_choices = vall_data['choice_0'].tolist() + vall_data['choice_1'].tolist() + vall_data['choice_2'].tolist()

In [35]:
# получаем векторные представления выборов ответов
train_choice_embs = create_embeds(train_choices,encoder,tokenizer,batch_size=128,max_length=512)
test_choices_embs = create_embeds(test_choices,encoder,tokenizer,batch_size=128,max_length=512)
vall_choices_embs = create_embeds(vall_choices,encoder,tokenizer,batch_size=128,max_length=512)

  0%|          | 0/172 [00:00<?, ?it/s]

  0%|          | 0/49 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [36]:
# Ищем вершины, наиболее схожие с ответами
faiss.normalize_L2(train_choice_embs)
D_ch_train,I_ch_train = index.search(train_choice_embs,k=1)
faiss.normalize_L2(test_choices_embs)
D_ch_test,I_ch_test = index.search(test_choices_embs,k=1)
faiss.normalize_L2(vall_choices_embs)
D_ch_val,I_ch_val = index.search(vall_choices_embs,k=1)

In [37]:
train_data['choice_1_concept'] = [concepts[j] for j in I_ch_train[:,0][:len(I_ch_train)//3]]
train_data['choice_2_concept'] = [concepts[j] for j in I_ch_train[:,0][len(I_ch_train)//3:2*len(I_ch_train)//3]]
train_data['choice_3_concept'] = [concepts[j] for j in I_ch_train[:,0][2*len(I_ch_train)//3:]]

In [38]:
test_data['choice_1_concept'] = [concepts[j] for j in I_ch_test[:,0][:len(I_ch_test)//3]]
test_data['choice_2_concept'] = [concepts[j] for j in I_ch_test[:,0][len(I_ch_test)//3:2*len(I_ch_test)//3]]
test_data['choice_3_concept'] = [concepts[j] for j in I_ch_test[:,0][2*len(I_ch_test)//3:]]

In [39]:
vall_data['choice_1_concept'] = [concepts[j] for j in I_ch_val[:,0][:len(I_ch_val)//3]]
vall_data['choice_2_concept'] = [concepts[j] for j in I_ch_val[:,0][len(I_ch_val)//3:2*len(I_ch_val)//3]]
vall_data['choice_3_concept'] = [concepts[j] for j in I_ch_val[:,0][2*len(I_ch_val)//3:]]

In [40]:
train_data[['choice_1_concept','choice_2_concept','choice_3_concept']].to_csv('./prep_data/train_choices.csv',index=False)
test_data[['choice_1_concept','choice_2_concept','choice_3_concept']].to_csv('./prep_data/test_choices.csv',index=False)
vall_data[['choice_1_concept','choice_2_concept','choice_3_concept']].to_csv('./prep_data/vall_choices.csv',index=False)

# Merge Candidates

In [41]:
import pandas as pd
import numpy as np
import ast

In [42]:
train_candidates = pd.DataFrame()
test_candidates = pd.DataFrame()
vall_candidates = pd.DataFrame()

In [43]:
train_chs = pd.read_csv('prep_data/train_choices.csv')
test_chs = pd.read_csv('prep_data/test_choices.csv')
vall_chs = pd.read_csv('prep_data/vall_choices.csv')

In [44]:
train_chs['choices'] = train_chs.apply(lambda x: list(x.values),axis=1)
test_chs['choices'] = test_chs.apply(lambda x: list(x.values),axis=1)
vall_chs['choices'] = vall_chs.apply(lambda x: list(x.values),axis=1)

In [45]:
train_lemma = pd.read_csv('prep_data/train_rank_by_ids.csv')
test_lemma = pd.read_csv('prep_data/test_rank_by_ids.csv')
vall_lemma = pd.read_csv('prep_data/vall_rank_by_ids.csv')

train_lemma['candidates_rank'] = train_lemma['candidates_rank'].apply(ast.literal_eval)
test_lemma['candidates_rank'] = test_lemma['candidates_rank'].apply(ast.literal_eval)
vall_lemma['candidates_rank'] = vall_lemma['candidates_rank'].apply(ast.literal_eval)

In [46]:
train_bge = pd.read_csv('prep_data/train_dream_bge.csv')
test_bge = pd.read_csv('prep_data/test_dream_bge.csv')
vall_bge = pd.read_csv('prep_data/val_dream_bge.csv')

train_bge['candidates'] = train_bge['candidates'].apply(ast.literal_eval)
test_bge['candidates'] = test_bge['candidates'].apply(ast.literal_eval)
vall_bge['candidates'] = vall_bge['candidates'].apply(ast.literal_eval)

In [47]:
train_candidates['candidates'] = train_lemma['candidates_rank'] + train_bge['candidates']
test_candidates['candidates'] =  test_lemma['candidates_rank'] + test_bge['candidates']
vall_candidates['candidates'] =  vall_lemma['candidates_rank'] + vall_bge['candidates'] 

In [48]:
candidates = {
    "train_candidates": train_candidates['candidates'].tolist(),
    "test_candidates": test_candidates['candidates'].tolist(),
    "vall_candidates": vall_candidates['candidates'].tolist(),
}

In [49]:
candidates['train_candidates'][0]

['serving_fish',
 'done_cooking',
 'actually',
 'wonderful',
 'glad',
 'interesting to cook',
 'good cook',
 'trying out new recipe',
 'said of food',
 'may find interesting recipe']